In [40]:
# stdlib imports
import os
from pathlib import Path
from datetime import timedelta

# external imports
import pandas as pd

In [36]:
RUNS = ['WCP', 'LEJOG']

# length of each run (miles)
LENGTHS = {
    'WCP': 870,
    'LEJOG': 874
}

# target lengths
TARGETS = [365,350,300,250,200,150,100]

# how many miles should we run each day for the given target?
RATES = {}
for run in RUNS:
    RATES[run] = {}
    for target in TARGETS:
        RATES[run][target] = LENGTHS[run]/target

In [27]:
# all the data is in the dropbox
DATA_FILE = Path.home() / 'Dropbox' / 'home' / 'shared' / 'martin' / 'WCP.xlsx'
DATA_FILE

PosixPath('/Users/martinchorley/Dropbox/home/shared/martin/WCP.xlsx')

In [28]:
# let's read in the run data
run_data = pd.read_excel(DATA_FILE, sheet_name=['WCP', 'LEJOG'], header=1)
run_data

{'WCP':      Day       Date  Mileage  Cumulative  Daily Rate  Remaining  \
 0      1 2020-06-15      NaN        0.00    0.000000     870.00   
 1      2 2020-06-16     6.29        6.29    3.145000     863.71   
 2      3 2020-06-17      NaN        6.29    2.096667     863.71   
 3      4 2020-06-18     3.71       10.00    2.500000     860.00   
 4      5 2020-06-19     2.85       12.85    2.570000     857.15   
 ..   ...        ...      ...         ...         ...        ...   
 361  362 2021-06-11      NaN      104.28    0.288066     765.72   
 362  363 2021-06-12      NaN      104.28    0.287273     765.72   
 363  364 2021-06-13      NaN      104.28    0.286484     765.72   
 364  365 2021-06-14      NaN      104.28    0.285699     765.72   
 365  366 2021-06-15      NaN      104.28    0.284918     765.72   
 
      Pace Required  Variance  
 0         2.390110 -2.390110  
 1         2.379366  0.765634  
 2         2.385939 -0.289273  
 3         2.382271  0.117729  
 4         2.38

In [30]:
# drop the days we didn't run (or haven't run yet)
for run in RUNS:
    run_data[run] = run_data[run].dropna()
    
run_data

{'WCP':     Day       Date  Mileage  Cumulative  Daily Rate  Remaining  Pace Required  \
 1     2 2020-06-16     6.29        6.29    3.145000     863.71       2.379366   
 3     4 2020-06-18     3.71       10.00    2.500000     860.00       2.382271   
 4     5 2020-06-19     2.85       12.85    2.570000     857.15       2.380972   
 5     6 2020-06-20     3.78       16.63    2.771667     853.37       2.377075   
 7     8 2020-06-22     9.92       26.55    3.318750     843.45       2.362605   
 9    10 2020-06-24     4.42       30.97    3.097000     839.03       2.363465   
 11   12 2020-06-26     7.52       38.49    3.207500     831.51       2.355552   
 13   14 2020-06-28     3.14       41.63    2.973571     828.37       2.360028   
 14   15 2020-06-29     1.12       42.75    2.850000     827.25       2.363571   
 15   16 2020-06-30    12.84       55.59    3.474375     814.41       2.333553   
 16   17 2020-07-01     1.10       56.69    3.334706     813.31       2.337098   
 17   18 

In [34]:
# how many miles have we got left to do, and how many are we doing on average every day

CURRENT_RATES = {}
REMAINING = {}

for run in RUNS:
    last_run = run_data[run].iloc[-1]
    CURRENT_RATES[run] = last_run['Cumulative']/last_run['Day']
    REMAINING[run] = LENGTHS[run] - last_run['Cumulative']
    
print(CURRENT_RATES)
print(REMAINING)

{'WCP': 3.0670588235294116, 'LEJOG': 3.7600000000000002}
{'WCP': 765.72, 'LEJOG': 851.44}


In [35]:
DAYS_TO_FINISH = {}
for run in RUNS:
    DAYS_TO_FINISH[run] = REMAINING[run] / CURRENT_RATES[run]
print(DAYS_TO_FINISH)

{'WCP': 249.65937859608746, 'LEJOG': 226.4468085106383}


In [46]:
PROJECTED_END = {}
for run in RUNS:
    first_run = run_data[run].iloc[0]
    start_date = first_run['Date']
    PROJECTED_END[run] = (start_date + timedelta(days=DAYS_TO_FINISH[run])).date()
print(PROJECTED_END)

{'WCP': datetime.date(2021, 2, 20), 'LEJOG': datetime.date(2021, 2, 24)}
